In [1]:
import os

In [2]:
%pwd

'/Users/rohansridhar/Desktop/BIKE-RENTAL/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rohansridhar/Desktop/BIKE-RENTAL'

In [5]:
from dataclasses import dataclass
from pathlib import Path 
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_n_estimators: int


In [6]:
from bikerental.constants import * 
from bikerental.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_n_estimators=self.params.n_estimators
        )
        return prepare_base_model_config

In [13]:
import os
import urllib.request as request
from zipfile import ZipFile
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pathlib import Path
from dataclasses import dataclass
import numpy as np


In [14]:

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.model = None
        self.regressor = None  # Added regressor attribute

    def save_model(self, path, model):
        # Save the model using joblib or any other preferred method
        pass

    def train_random_forest(self, x_train, y_train, x_test, y_test):
        self.regressor = RandomForestRegressor(n_estimators=self.config.params_n_estimators)
        self.regressor.fit(x_train, y_train)

        # Make predictions on the test set
        y_pred_test = self.regressor.predict(x_test)

        # Calculate R-squared score for the test set
        r2_test = r2_score(y_test, y_pred_test)
        print(f"R-squared score on test set: {r2_test:.4f}")

    def update_base_model(self, x_train, y_train):
        # Extract individual decision tree
        tree_estimator = self.regressor.estimators_[0]

        # Make predictions using the decision tree
        tree_predictions = tree_estimator.predict(x_train)
        tree_predictions = tree_predictions.reshape(-1, 1)

        # Use tree predictions as features for the final linear model
        self.model = LinearRegression()
        self.model.fit(tree_predictions, y_train)

        # Print the coefficients if needed
        print("Coefficients:", self.model.coef_)

    def save_updated_model(self):
        # Save the updated model using joblib or any other preferred method
        pass


# Load and preprocess data
day = pd.read_csv("artifacts/data_ingestion/day.csv")
day.rename(columns={'instant': 'rec_id', 'dteday': 'datetime', 'yr': 'year', 'mnth': 'month',
                    'weathersit': 'weather_condition', 'hum': 'humidity', 'cnt': 'total_count'}, inplace=True)
day = day.drop(columns=['rec_id', 'datetime', 'atemp', 'casual', 'registered'])
x = day.drop(columns='total_count')
y = day['total_count']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [15]:
# Example usage
try:
    config_manager = ConfigurationManager()
    prepare_base_model_config = config_manager.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.train_random_forest(x_train, y_train, x_test, y_test)  # Call train_random_forest first
    prepare_base_model.update_base_model(x_train, y_train)
    prepare_base_model.save_model(path=prepare_base_model_config.updated_base_model_path, model=prepare_base_model)
except Exception as e:
    raise e


[2024-01-17 14:50:52,339: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-17 14:50:52,343: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-17 14:50:52,344: INFO: common: created directory at: artifacts]
[2024-01-17 14:50:52,344: INFO: common: created directory at: artifacts/prepare_base_model]
R-squared score on test set: 0.8594
Coefficients: [0.9389959]
